# Experiment 002: Enhanced Features with TF-IDF and Heading Analysis

This experiment adds TF-IDF features and heading-level features to improve upon the baseline.

**Strategy:** Priority 1 - Enhanced Feature Engineering

**Expected improvements:**
- TF-IDF features for semantic content patterns
- Heading level features to capture hierarchy
- Semantic position features for first/last cell patterns
- Notebook-level features for context

**Validation:** 5-fold GroupKFold with notebook_id groups, Kendall tau metric

In [1]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from tqdm import tqdm
import re
from collections import Counter
from sklearn.model_selection import GroupKFold
from sklearn.metrics import make_scorer
from scipy.stats import kendalltau
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

# Set paths
TRAIN_PATH = Path('/home/data/train')
TEST_PATH = Path('/home/data/test')
ORDERS_PATH = Path('/home/data/train_orders.csv')

print("Loading data...")
orders_df = pd.read_csv(ORDERS_PATH)
print(f"Orders shape: {orders_df.shape}")
print(f"Unique notebooks: {orders_df['id'].nunique()}")

Loading data...


Orders shape: (119256, 2)
Unique notebooks: 119256


In [3]:
# Load a subset of training data for faster iteration
# Use 5000 notebooks for this experiment
train_notebooks = orders_df['id'].unique()[:5000]
print(f"Using {len(train_notebooks)} notebooks for training")

# Create training data
all_cells = []
notebook_sizes = {}

for notebook_id in tqdm(train_notebooks, desc="Loading training notebooks"):
    notebook_path = TRAIN_PATH / f"{notebook_id}.json"
    
    with open(notebook_path, 'r') as f:
        notebook = json.load(f)
    
    cell_order = orders_df[orders_df['id'] == notebook_id]['cell_order'].iloc[0].split()
    cell_positions = {cell_id: pos for pos, cell_id in enumerate(cell_order)}
    
    notebook_sizes[notebook_id] = len(cell_order)
    
    # The notebook structure has cell_type and source as separate dictionaries
    cell_types = notebook['cell_type']
    sources = notebook['source']
    
    for cell_id in cell_types.keys():
        cell_type = cell_types[cell_id]
        source = sources[cell_id]
        
        all_cells.append({
            'notebook_id': notebook_id,
            'cell_id': cell_id,
            'cell_type': cell_type,
            'source': source,
            'position': cell_positions[cell_id]
        })

train_df = pd.DataFrame(all_cells)
print(f"Training data shape: {train_df.shape}")
print(f"Average cells per notebook: {train_df.groupby('notebook_id').size().mean():.1f}")

Using 5000 notebooks for training


Loading training notebooks:   0%|          | 0/5000 [00:00<?, ?it/s]

Loading training notebooks:   0%|          | 10/5000 [00:00<00:51, 97.21it/s]

Loading training notebooks:   0%|          | 20/5000 [00:00<00:50, 98.26it/s]

Loading training notebooks:   1%|          | 31/5000 [00:00<00:49, 99.45it/s]

...

Loading training notebooks: 100%|█████████▉| 4975/5000 [00:51<00:00, 103.38it/s]

Loading training notebooks: 100%|█████████▉| 4986/5000 [00:51<00:00, 102.74it/s]

Loading training notebooks: 100%|█████████▉| 4997/5000 [00:51<00:00, 102.49it/s]

Loading training notebooks: 100%|██████████| 5000/5000 [00:51<00:00, 96.53it/s] 

Training data shape: (233581, 5)
Average cells per notebook: 46.7


In [4]:
# Enhanced feature extraction function
def extract_enhanced_features(df):
    """Extract enhanced features including TF-IDF and heading analysis"""
    
    # Basic text features
    df['source_length'] = df['source'].str.len()
    df['line_count'] = df['source'].str.count('\\n') + 1
    df['word_count'] = df['source'].str.split().str.len()
    df['char_count'] = df['source'].str.replace('\\s+', '', regex=True).str.len()
    
    # Binary flags
    df['has_import'] = df['source'].str.contains('import ', case=False, na=False).astype(int)
    df['has_comment'] = df['source'].str.contains('#', na=False).astype(int)
    df['has_heading'] = df['source'].str.contains('^#+\\s', regex=True, na=False).astype(int)
    df['has_code_block'] = df['source'].str.contains('```', na=False).astype(int)
    df['has_link'] = df['source'].str.contains('\\[.*\\]\\(http', regex=True, na=False).astype(int)
    
    # Cell type
    df['cell_type_code'] = (df['cell_type'] == 'code').astype(int)
    
    # Heading level analysis
    df['heading_level'] = 0
    for level in range(1, 7):
        mask = df['source'].str.match(f'^#{{{level}}}\\s', na=False)
        df.loc[mask, 'heading_level'] = level
    
    # Common heading text features
    common_headings = ['introduction', 'conclusion', 'eda', 'exploratory data analysis', 
                       'model', 'results', 'analysis', 'data', 'preprocessing', 
                       'visualization', 'plot', 'train', 'test', 'validation']
    
    for heading in common_headings:
        df[f'heading_{heading}'] = df['source'].str.contains(heading, case=False, na=False).astype(int)
    
    # Semantic position features
    df['has_print'] = df['source'].str.contains('print\\s*\\(', na=False).astype(int)
    df['has_kaggle'] = df['source'].str.contains('kaggle', case=False, na=False).astype(int)
    df['has_input'] = df['source'].str.contains('input', case=False, na=False).astype(int)
    df['has_data'] = df['source'].str.contains('\\bdata\\b', case=False, na=False).astype(int)
    df['has_function'] = df['source'].str.contains('def\\s+\\w+\\s*\\(', regex=True, na=False).astype(int)
    df['has_class'] = df['source'].str.contains('class\\s+\\w+\\s*\\(', regex=True, na=False).astype(int)
    df['has_plot'] = df['source'].str.contains('\\.plot\\s*\\(|\\.show\\s*\\(|plt\\.', regex=True, na=False).astype(int)
    
    # First/last cell indicators
    df['likely_first_cell'] = ((df['has_import'] == 1) | (df['has_kaggle'] == 1) | (df['has_input'] == 1)).astype(int)
    df['likely_last_cell'] = ((df['has_print'] == 1) | (df['has_plot'] == 1)).astype(int)
    
    return df

print("Extracting enhanced features...")
train_df = extract_enhanced_features(train_df)
print(f"Features extracted. Shape: {train_df.shape}")

Extracting enhanced features...


Features extracted. Shape: (233581, 39)


In [5]:
# Add notebook-level features
print("Adding notebook-level features...")

# Calculate notebook-level statistics
notebook_stats = train_df.groupby('notebook_id').agg({
    'source_length': ['mean', 'std'],
    'word_count': ['mean', 'std'],
    'cell_type_code': 'mean',  # code ratio
    'position': 'max'  # notebook size
}).round(2)

notebook_stats.columns = ['_'.join(col).strip() for col in notebook_stats.columns]
notebook_stats = notebook_stats.reset_index()
notebook_stats.rename(columns={'position_max': 'notebook_size'}, inplace=True)

# Merge notebook-level features
train_df = train_df.merge(notebook_stats, on='notebook_id', how='left')

# Add relative position feature (percentile within notebook)
train_df['relative_position'] = train_df['position'] / train_df['notebook_size']

print(f"Final training shape: {train_df.shape}")
print(f"Notebook-level features added: {list(notebook_stats.columns)}")

Adding notebook-level features...


Final training shape: (233581, 46)
Notebook-level features added: ['notebook_id', 'source_length_mean', 'source_length_std', 'word_count_mean', 'word_count_std', 'cell_type_code_mean', 'notebook_size']


In [6]:
# TF-IDF features for semantic content
print("Extracting TF-IDF features...")

# Get text from markdown cells for TF-IDF
markdown_texts = train_df[train_df['cell_type'] == 'markdown']['source'].fillna('').tolist()

# Use a subset of terms for efficiency
max_features = 1000
vectorizer = TfidfVectorizer(
    max_features=max_features,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.95
)

# Fit TF-IDF on markdown texts
vectorizer.fit(markdown_texts)
print(f"TF-IDF vocabulary size: {len(vectorizer.vocabulary_)}")

# Transform all texts (both code and markdown)
all_texts = train_df['source'].fillna('').tolist()
tfidf_matrix = vectorizer.transform(all_texts)

# Add TF-IDF features to dataframe
tfidf_feature_names = [f'tfidf_{i}' for i in range(max_features)]
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_feature_names,
    index=train_df.index
)

# Concatenate with original dataframe
train_df = pd.concat([train_df, tfidf_df], axis=1)

print(f"Final shape with TF-IDF: {train_df.shape}")
print(f"TF-IDF features added: {max_features}")

Extracting TF-IDF features...


TF-IDF vocabulary size: 1000


Final shape with TF-IDF: (233581, 1046)
TF-IDF features added: 1000


In [7]:
# Prepare feature columns
basic_features = ['source_length', 'line_count', 'word_count', 'char_count', 
                  'has_import', 'has_comment', 'has_heading', 'has_code_block', 
                  'has_link', 'cell_type_code', 'heading_level']

heading_features = [f'heading_{h}' for h in ['introduction', 'conclusion', 'eda', 
                    'exploratory data analysis', 'model', 'results', 'analysis', 
                    'data', 'preprocessing', 'visualization', 'plot', 'train', 
                    'test', 'validation']]

semantic_features = ['has_print', 'has_kaggle', 'has_input', 'has_data', 
                     'has_function', 'has_class', 'has_plot', 'likely_first_cell', 
                     'likely_last_cell']

notebook_features = ['source_length_mean', 'source_length_std', 'word_count_mean', 
                     'word_count_std', 'cell_type_code_mean', 'notebook_size']

tfidf_features = tfidf_feature_names

feature_cols = (basic_features + heading_features + semantic_features + 
                notebook_features + tfidf_features + ['relative_position'])

print(f"Total features: {len(feature_cols)}")
print(f"Basic features: {len(basic_features)}")
print(f"Heading features: {len(heading_features)}")
print(f"Semantic features: {len(semantic_features)}")
print(f"Notebook features: {len(notebook_features)}")
print(f"TF-IDF features: {len(tfidf_features)}")

Total features: 1041
Basic features: 11
Heading features: 14
Semantic features: 9
Notebook features: 6
TF-IDF features: 1000


In [8]:
# Define evaluation metric
def kendall_tau_score(y_true, y_pred):
    """Calculate Kendall tau correlation"""
    return kendalltau(y_true, y_pred).correlation

kendall_scorer = make_scorer(kendall_tau_score, greater_is_better=True)

# Cross-validation setup
gkf = GroupKFold(n_splits=5)
groups = train_df['notebook_id']

X = train_df[feature_cols]
y = train_df['position']

print("Starting cross-validation...")
cv_scores = []

for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Train model
    model = lgb.LGBMRegressor(
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    # Predict on validation set
    y_pred = model.predict(X_val)
    
    # Calculate per-notebook Kendall tau
    val_df = train_df.iloc[val_idx].copy()
    val_df['pred_position'] = y_pred
    
    fold_scores = []
    for notebook_id in val_df['notebook_id'].unique():
        notebook_data = val_df[val_df['notebook_id'] == notebook_id]
        if len(notebook_data) > 1:
            score = kendall_tau_score(
                notebook_data['position'].values,
                notebook_data['pred_position'].values
            )
            fold_scores.append(score)
    
    fold_score = np.mean(fold_scores)
    cv_scores.append(fold_score)
    print(f"Fold {fold + 1}: {fold_score:.4f}")

print(f"\nCV Score: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

Starting cross-validation...


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.271855 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 184263
[LightGBM] [Info] Number of data points in the train set: 186865, number of used features: 1041
[LightGBM] [Info] Start training from score 40.286688


Fold 1: 0.9951


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.351872 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 184229
[LightGBM] [Info] Number of data points in the train set: 186865, number of used features: 1041
[LightGBM] [Info] Start training from score 41.117052


Fold 4: 0.9950


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.276111 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 185604
[LightGBM] [Info] Number of data points in the train set: 186864, number of used features: 1041
[LightGBM] [Info] Start training from score 41.118685


Fold 5: 0.9919

CV Score: 0.9936 ± 0.0015


In [11]:
# Train final model on all data
print("Training final model on all data...")

final_model = lgb.LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

final_model.fit(X, y)

# Feature importance
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 most important features:")
print(importance_df.head(20))

Training final model on all data...


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.323158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 190063
[LightGBM] [Info] Number of data points in the train set: 233581, number of used features: 1041
[LightGBM] [Info] Start training from score 40.946348



Top 20 most important features:
                  feature  importance
1040    relative_position        2690
39          notebook_size        2395
34     source_length_mean         755
37         word_count_std         634
35      source_length_std         507
36        word_count_mean         449
38    cell_type_code_mean         398
770             tfidf_730         139
282             tfidf_242          90
32      likely_first_cell          75
15          heading_model          65
211             tfidf_171          60
23           heading_test          51
43                tfidf_3          35
253             tfidf_213          35
947             tfidf_907          34
0           source_length          32
364             tfidf_324          30
3              char_count          27
2              word_count          26


In [12]:
# Prediction function for test data
def predict_notebook_order(notebook_id, path, model, feature_cols, vectorizer):
    """Predict cell order for a notebook"""
    notebook_path = path / f"{notebook_id}.json"
    
    with open(notebook_path, 'r') as f:
        notebook = json.load(f)
    
    # Extract features - handle the same structure as training data
    cells = []
    cell_ids = []
    
    # The notebook structure has cell_type and source as separate dictionaries
    cell_types = notebook['cell_type']
    sources = notebook['source']
    
    for cell_id in cell_types.keys():
        cell_type = cell_types[cell_id]
        source = sources[cell_id]
        
        cells.append({
            'cell_id': cell_id,
            'cell_type': cell_type,
            'source': source
        })
        cell_ids.append(cell_id)
    
    features_df = pd.DataFrame(cells)
    
    # Extract enhanced features
    features_df = extract_enhanced_features(features_df)
    
    # Add placeholder for notebook-level features (will be filled)
    features_df['source_length_mean'] = features_df['source_length'].mean()
    features_df['source_length_std'] = features_df['source_length'].std()
    features_df['word_count_mean'] = features_df['word_count'].mean()
    features_df['word_count_std'] = features_df['word_count'].std()
    features_df['cell_type_code_mean'] = features_df['cell_type_code'].mean()
    features_df['notebook_size'] = len(features_df)
    
    # Add relative position (placeholder, will be predicted)
    features_df['relative_position'] = 0.5  # placeholder
    
    # Transform TF-IDF features
    all_texts = features_df['source'].fillna('').tolist()
    tfidf_matrix = vectorizer.transform(all_texts)
    
    tfidf_feature_names = [f'tfidf_{i}' for i in range(1000)]
    tfidf_df = pd.DataFrame(
        tfidf_matrix.toarray(),
        columns=tfidf_feature_names,
        index=features_df.index
    )
    
    features_df = pd.concat([features_df, tfidf_df], axis=1)
    
    # Ensure all feature columns exist
    for col in feature_cols:
        if col not in features_df.columns:
            features_df[col] = 0
    
    # Predict positions
    X_test = features_df[feature_cols]
    predictions = model.predict(X_test)
    
    # Sort by predicted position
    features_df['predicted_position'] = predictions
    features_df = features_df.sort_values('predicted_position')
    
    # Return ordered cell IDs as space-separated string
    ordered_cells = features_df['cell_id'].tolist()
    return ' '.join(ordered_cells)

In [13]:
# Generate submission for all test notebooks
print("Generating submission for all test notebooks...")

test_notebooks = [f.stem for f in TEST_PATH.glob('*.json')]
submission_data = []

for notebook_id in tqdm(test_notebooks, desc="Predicting notebooks"):
    try:
        predicted_order = predict_notebook_order(notebook_id, TEST_PATH, final_model, feature_cols, vectorizer)
        submission_data.append({
            'id': notebook_id,
            'cell_order': predicted_order
        })
    except Exception as e:
        print(f"Error with {notebook_id}: {e}")

submission_df = pd.DataFrame(submission_data)
print(f"Submission shape: {submission_df.shape}")
print(submission_df.head())

Generating submission for all test notebooks...


Predicting notebooks:   0%|          | 0/20000 [00:00<?, ?it/s]

Predicting notebooks:   0%|          | 3/20000 [00:00<14:29, 22.99it/s]

Predicting notebooks:   0%|          | 6/20000 [00:00<14:15, 23.38it/s]

Predicting notebooks:   0%|          | 9/20000 [00:00<13:12, 25.24it/s]

...

Predicting notebooks: 100%|█████████▉| 19990/20000 [12:52<00:00, 26.17it/s]

Predicting notebooks: 100%|█████████▉| 19993/20000 [12:52<00:00, 26.72it/s]

Predicting notebooks: 100%|█████████▉| 19997/20000 [12:52<00:00, 27.71it/s]

Predicting notebooks: 100%|██████████| 20000/20000 [12:52<00:00, 25.88it/s]

Submission shape: (20000, 2)
               id                                         cell_order
0  a283f88578d447  e031fdc5 e04e3c89 29afef3c 4051523d bc48d183 a...
1  4b5781d84d4f04  1898e18c 781309ac def9cf6d 2ebababb 9dff4a05 0...
2  032e1e8598af69  b900c928 cd8da9d0 6c301d46 c054583f 2b10ddf8 7...
3  598969c8543546  71472504 b5fdf2ce 4d9eb115 ff8abe04 cd4e2158 b...
4  6986f993569b86  04e4e828 7260068e 58e98092 328ec93c 0eed5fa5 b...


In [14]:
# Save submission
submission_path = '/home/submission/submission_002.csv'
submission_df.to_csv(submission_path, index=False)
print(f"Submission saved to {submission_path}")

# Verify format
sample_sub = pd.read_csv('/home/data/sample_submission.csv')
print(f"\nColumns match: {list(submission_df.columns) == list(sample_sub.columns)}")
print(f"Number of rows match: {len(submission_df) == len(sample_sub)}")

Submission saved to /home/submission/submission_002.csv

Columns match: True
Number of rows match: True
